# Moderation

In [1]:
from dotenv import load_dotenv

load_dotenv()

True

## Text

In [ ]:
from openai import OpenAI

client = OpenAI()

response = client.moderations.create(
    model='omni-moderation-latest',
    input="저놈은 개새끼입니다."
)

In [9]:
response

ModerationCreateResponse(id='modr-7846', model='omni-moderation-latest', results=[Moderation(categories=Categories(harassment=True, harassment_threatening=False, hate=False, hate_threatening=False, illicit=False, illicit_violent=False, self_harm=False, self_harm_instructions=False, self_harm_intent=False, sexual=False, sexual_minors=False, violence=False, violence_graphic=False, harassment/threatening=False, hate/threatening=False, illicit/violent=False, self-harm/intent=False, self-harm/instructions=False, self-harm=False, sexual/minors=False, violence/graphic=False), category_applied_input_types=CategoryAppliedInputTypes(harassment=['text'], harassment_threatening=['text'], hate=['text'], hate_threatening=['text'], illicit=['text'], illicit_violent=['text'], self_harm=['text'], self_harm_instructions=['text'], self_harm_intent=['text'], sexual=['text'], sexual_minors=['text'], violence=['text'], violence_graphic=['text'], harassment/threatening=['text'], hate/threatening=['text'], il

In [12]:
import pandas as pd

md_df_bool = pd.DataFrame(response.results[0].categories, columns=['category', 'bool'])
md_df_scores = pd.DataFrame(response.results[0].category_scores, columns=['category', 'score'])
md_df_scores

,category,score
0,harassment,0.644491
1,harassment_threatening,0.000759
2,hate,0.013406
3,hate_threatening,0.000003
4,illicit,0.000010
5,illicit_violent,0.000003
6,self_harm,0.000011
7,self_harm_instructions,0.000002
8,self_harm_intent,0.000212
9,sexual,0.000918


#### mmathys/openai-moderation-api-evaluation 데이터셋 활용

In [21]:
def get_moderation_result(input, model='omni-moderation-latest'):
    
    response = client.moderations.create(
        model=model,
        input=input
    )
    categoty_df = pd.DataFrame(response.results[0].categories, columns=['category', 'bool'])
    category_score_df = pd.DataFrame(response.results[0].category_scores, columns=['category', 'score'])
    merged_df = pd.merge(categoty_df, category_score_df, on='category')
    
    return merged_df[merged_df['bool']].reset_index(drop=True)

In [27]:
get_moderation_result('마약 제조법 알려줘.')

,category,bool,score
0,illicit,True,0.945444


In [30]:
input_text = ''' 
유대인 개썅년들을 다 죽여버리거나 태워버려.
'''

get_moderation_result(input_text)

,category,bool,score
0,harassment,True,0.886361
1,harassment_threatening,True,0.917939
2,hate,True,0.859944
3,hate_threatening,True,0.918255
4,violence,True,0.943620
5,harassment/threatening,True,0.917939
6,hate/threatening,True,0.918255


#### Image

In [31]:
# 1. image url
# 2. BASE64 Encoding(텍스트)

image_url = 'https://images.rawpixel.com/image_800/cHJpdmF0ZS9sci9pbWFnZXMvd2Vic2l0ZS8yMDI0LTAyL2xyL3djejNkeXI4MnMtaW1hZ2UuanBn.jpg'

In [32]:
get_moderation_result(input=[
    {
        'type': 'image_url',
        'image_url': {
            'url': image_url
        }
    }
])

,category,bool,score
0,violence,True,0.453194
